**Extract more Data from TMDB!**

* There is no financial information included in the IMDB data (e.g. budget or revenue).

* This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before you will be able to determine which movies are successful.

* The Movie Database (TMDB) as a great source of financial data (https://www.themoviedb.org/) and offers a free API for programmatic access to their data!

* My stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

**Specifications**

* My stakeholder would like me to extract and save the results for movies that meet all of the criteria established in the first part of the project.

* As a proof-of-concept, they requested I perform a test extraction of movies that started in 2000 or 2001

* Each year should be saved as a separate .csv.gz file

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm_notebook

In [16]:
import json
with open('/Users/oleksandraaliyeva/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['client-id', 'api-key'])

In [17]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [34]:
#create folder for data
import os
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2006.csv.gz',
 'tmdb_api_results_2010.json',
 'final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'tmdb_api_results_2006.json',
 '.DS_Store',
 'final_tmdb_data_2008.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'tmdb_results_combined_2000_2001.csv.gz',
 'final_tmdb_data_2004.csv.gz',
 'tmdb_api_results_2007.json',
 'final_tmdb_data_2020.csv.gz',
 'tmdb_api_results_2011.json',
 'tmdb_api_results_2020.json',
 'tmdb_api_results_2016.json',
 'basics.csv.gz',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'tmdb_api_results_2001.json',
 'ratings.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'tmdb_api_results_2017.json',
 'tmdb_api_results_2021.json',
 'tmdb_api_results_2018.json',
 'tmdb_api_results_2002.json',
 'final_tmdb_data_2019.csv.gz',
 'final_tmdb_data_2007.csv.gz',
 'tmdb_api_results_2014.json',
 'final_tmdb_data_2015.csv.gz',
 'tmdb_api_results_2015.json',
 'final_tmdb_data_2021.c

In [19]:
#creat instance for years
YEARS_TO_GET = list(range(2000,2022))

In [20]:
#function to get movie rating
def get_movie_with_rating(movie_id): 
    
    #get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    
    #loop through countries in releases
    for c in releases['countries']:
        #if the country abbreviation==US
        if c['iso_3166_1'] =='US':
            #save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
            
    return movie_info

In [21]:
#function to create a new json file 
def write_json(new_data, filename): 
        """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
        
        with open(filename,'r+') as file:
            #first we load existing data into a dict.
            file_data = json.load(file)
            
            #choose extend or append
            if (type(new_data) == list) & (type(file_data) == list):
                file_data.extend(new_data)
            else:
                file_data.append(new_data)
                
        #sets file's current position at offset.
            file.seek(0)
            
        #convert back to json.
            json.dump(file_data, file)

In [22]:
#start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    
    #defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    #if it does not exist: create it
    if file_exists == False:
        
        #save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    #load in the dataframe from project part 1 as basics:
    basics = pd.read_csv('Data/basics.csv.gz')
    
    #saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    
    #saving movie ids to list
    movie_ids = df['tconst'].copy()
   
    #load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    #INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        #attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id) 
        
            #append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            
            #short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        #if it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue
      
    #open json file and save as csv
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/22 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/207 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/241 [00:00<?, ?it/s]

Movies from 2002:   0%|          | 0/288 [00:00<?, ?it/s]

Movies from 2003:   0%|          | 0/367 [00:00<?, ?it/s]

...

Movies from 2017:   0%|          | 0/1218 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/1108 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/1005 [00:00<?, ?it/s]

Movies from 2020:   0%|          | 0/1057 [00:00<?, ?it/s]

Movies from 2021:   0%|          | 0/1186 [00:00<?, ?it/s]

In [3]:
#read csv file for 2000
y_2000 = pd.read_csv('Data/final_tmdb_data_2000.csv.gz')

#drop first row and check the head
y_2000 = y_2000.iloc[1: , :]
y_2000.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.4,21.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,4.4,7.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,0.0,0.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.1,1859.0,PG
5,tt0118852,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,49511.0,en,Chinese Coffee,...,0.0,99.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,There's a fine line between friendship and bet...,Chinese Coffee,0.0,6.9,45.0,R


In [4]:
#read csv file for 2000
y_2001 = pd.read_csv('Data/final_tmdb_data_2001.csv.gz')

#drop first row and check the head
y_2001 = y_2001.iloc[1: , :]
y_2001.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
1,tt0035423,0.0,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,NaN,48000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",NaN,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,If they lived in the same century they'd be pe...,Kate & Leopold,0.0,6.3,1102.0,PG-13
2,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.0,3.0,NaN
3,tt0118589,0.0,/bLntSfsqUheiUirdV9Ts8znqsTM.jpg,NaN,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,10696.0,en,Glitter,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.6,115.0,PG-13
4,tt0118652,0.0,/mWxJEFRMvkG4UItYJkRDMgWQ08Y.jpg,NaN,1000000.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...",NaN,17140.0,en,The Attic Expeditions,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,His search for peace of mind... will leave his...,The Attic Expeditions,0.0,5.0,27.0,R
5,tt0119004,0.0,/7xrlSPGDO4CDT6IHTctDlkYxTzw.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,37857.0,en,Don's Plum,...,6297.0,89.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Don's Plum,0.0,5.4,62.0,NaN


In [5]:
#concatenate the previous results and new results. 
merged = pd.concat([y_2000, y_2001],
                      ignore_index=True)
display(merged.head(3), merged.tail(3))

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.4,21.0,NaN
1,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,4.4,7.0,NaN
2,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,0.0,0.0,NaN


,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
2469,tt8795764,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",https://www.utahwolf.com/films/coming-soon-new...,871624.0,en,New Breed,...,0.0,57.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,New Breed,0.0,0.0,0.0,NR
2470,tt9071078,0.0,NaN,NaN,0.0,"[{'id': 28, 'name': 'Action'}]",http://www.hkcinemagic.com/en/movie.asp?id=6627,201706.0,cn,致命密函,...,0.0,90.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,NaN,Chinese Heroes,0.0,3.0,2.0,NaN
2471,tt0274057,0.0,NaN,NaN,1000000.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,968521.0,en,Robbie's Brother,...,0.0,88.0,[],Released,NaN,Robbie's Brother,0.0,0.0,0.0,NaN


In [6]:
merged.to_csv("Data/tmdb_results_combined_2000_2001.csv.gz", 
              compression="gzip", index=False)

In [7]:
#add the data from years 2002-2022
YEARS_TO_GET = list(range(2002, 2022))
for YEAR in YEARS_TO_GET:
    df = pd.read_csv(f"Data/final_tmdb_data_{YEAR}.csv.gz", 
                     lineterminator='\n')
    merged = pd.concat([merged, df])

In [31]:
#create one file with all data from API
merged.to_csv("Data/tmdb_results_combined.csv.gz", compression="gzip", 
              index=False)